In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import pandas as pd
import rasterio.features
from affine import Affine

In [ ]:
# Reading Data
gdf = gpd.read_file("C://Users//27784//cooling degree days-2025-2-10//tas_day_AWI-CM-1-1-MR_ssp126//国界和省级分界//省级//2024年省级.shp")
nc_data = xr.open_dataset("C://Users//27784//HDD CDD 20250421//HDD CDD_year_multi_model_mean//CDD_multi_model_mean_ssp126.nc")
cdd = nc_data['CDD_mean']

In [ ]:
# Generate mask
lon = cdd.lon.values
lat = cdd.lat.values
dx = (lon[-1] - lon[0]) / (len(lon) - 1)
dy = (lat[-1] - lat[0]) / (len(lat) - 1)
transform = Affine.translation(lon[0] - dx/2, lat[0] - dy/2) * Affine.scale(dx, dy)

masks = []
for geom in gdf.geometry:
    mask = rasterio.features.rasterize(
        [(geom, 1)],
        out_shape=(len(lat), len(lon)),
        transform=transform,
        fill=0,
        dtype=np.uint8,
        all_touched=True
    )
    masks.append(mask.astype(bool))

In [ ]:
# Calculate the mean value
mask_da = xr.DataArray(np.stack(masks), dims=('province', 'lat', 'lon'), coords={'province': gdf.index})
cdd_masked = cdd.where(mask_da)
cdd_means = cdd_masked.mean(dim=['lat', 'lon'], skipna=True)


In [ ]:
# ---------------------Generate CSV file--------------------
# ---------------------------------------------

# Define the half-width of the neighborhood window (e.g., half-width=2 means 2 years before and after, totaling 5 years)
window_half_width = 5  # Can be adjusted according to requirements

# Create a list of target years (every 5 years, e.g., 2015, 2020, ..., 2100)
target_years = range(2015, 2101, 5)

# Create an empty DataFrame with initial column containing province names
df_all_years = pd.DataFrame({'province': gdf['ENG_NAME']})

# Loop through each target year
for year in target_years:
    # Calculate the start and end years of the window (considering data boundaries)
    start_year = max(year - window_half_width, cdd_means['year'].min().item())
    end_year = min(year + window_half_width, cdd_means['year'].max().item())
    
    # Extract all CDD data within the window
    cdd_window = cdd_means.sel(year=slice(start_year, end_year))
    
    # Calculate the mean within the window (along the time dimension)
    cdd_mean = cdd_window.mean(dim='year')
    
    # Add the result to the DataFrame
    df_all_years[f'{year}'] = cdd_mean.values

# --- Sort by the specified province order ---
# Define the province order
province_order = [
    'Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Guangdong', 
    'Gansu', 'Guangxi', 'Guizhou', 'Henan', 'Hubei', 
    'Hebei', 'Hainan', 'Heilongjiang', 'Hunan', 'Jilin', 
    'Jiangsu', 'Jiangxi', 'Liaoning', 'Neimenggu', 'Ningxia', 
    'Qinghai', 'Sichuan', 'Shandong', 'Shanghai', 'Shaanxi', 
    'Shanxi', 'Tianjin', 'Xinjiang', 'Xizang', 'Yunnan', 
    'Zhejiang'
]

# Convert the province column to categorical type (sorted by the specified order)
df_all_years['province'] = pd.Categorical(
    df_all_years['province'], 
    categories=province_order, 
    ordered=True
)

# Sort by province and reset the index
df_sorted = df_all_years.sort_values('province').reset_index(drop=True)

# --- Save as CSV file ---
df_sorted.to_csv('CDD_multi_model_province_mean_ssp126.csv', index=False, encoding='utf-8-sig')
